In [25]:
# all imports 
%pylab inline

import sys
sys.path.append("/root")
sys.path.append("/home/peter")
sys.path.append("/home/peter/Desktop")
sys.path.append("/home/peter/Desktop/projects")
sys.path.append("/home/peter/Desktop/work")
sys.path.append("/home/peter/code/projects")
sys.path.append("/home/peter/code/work")
sys.path.append("/home/peter/code/common")

import os
import socket
from ipywidgets import interact, interactive, fixed, IntSlider, FloatSlider
from IPython.display import clear_output, display, HTML
import cv2
import os
import pickle as pkl
from sklearn import linear_model
from sklearn.decomposition import PCA
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
import theano.tensor as T
import theano as th
from aidevutil import *
import cymysql as sql
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import datetime
import time
from sklearn.preprocessing import scale, robust_scale, normalize
from trading_env import TradingEnv
from skrules import SkopeRules
from sklearn.metrics import precision_recall_curve
# Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
import sklearn as sk
import time

In [26]:
input_source = np.load(open('data_btc_1h.npy','rb'))
to_predict = np.load(open('data_btc_1h_targets.npy','rb'))

to_predict = to_predict[3,:].reshape(-1)

In [27]:
input_source.shape

In [28]:
input_source = input_source.T

In [29]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [30]:
test_input_source = test_input_source.T
input_source = input_source.T

In [31]:
input_source.shape, test_input_source.shape

In [32]:
to_predict[0:50]

In [33]:
winlen = 1

In [34]:
sliding_window_jump = 1
predict_time_ahead = 1

In [35]:
binary = 1

In [36]:
def get_partitioned_counts(cl, rr, rt):
    " data, steps_forward, partition_num"
    cl = cl[rr:] - cl[0:-rr]
    cl = hstack([[0]*rr, cl])
    pdown = len(cl[cl < -rt])
    pup = len(cl[cl > rt])
    pmid = len(cl[(cl <= rt) & (cl >= -rt)])
    return pup, pmid, pdown

In [37]:
def partition_data_num(cl, predict_ahead):
    a=[]
    b=[]
    c=[]
    ts = np.linspace(0.001, 10.0, 100000, )
    for x in tqdm(ts):
        au, am, ad = get_partitioned_counts(cl, predict_ahead, x)
        a.append(au)
        b.append(am)
        c.append(ad)
    a, b, c = array(a), array(b), array(c)
    return ts[np.argmin(np.abs(a-b) + np.abs(a-c) + np.abs(b-c))]

In [38]:
# create training set
if not binary:
    sep = partition_data_num(to_predict, predict_time_ahead)
    sep

In [39]:
#@jit
def prepare_data(input_source, to_predict, binary=1):

    x = []
    y = []

    ups = []
    downs = []
    mids = []

    for i in range(0, input_source.shape[1]-(winlen+100), sliding_window_jump):
        # form the input
        txs = input_source[:, i:i+winlen]#.astype(np.float64)
        #xs = scale(xs, axis=1)
        xs = txs.ravel()

        # for the output
        now = to_predict[i+winlen-1]# close
        future = to_predict[i+winlen+(predict_time_ahead-1)] # next close

        ys = future-now
        magn = abs(ys)

        if not binary:
            if magn < sep:
                mids.append( (xs, (np.array([1]))) )
            else:
                if ys < 0:
                    downs.append( (xs, (np.array([2]))) )
                else:
                    ups.append( (xs, (np.array([0]))) )
        else:
            if ys <= 0:
                downs.append( (xs, (np.array([1]))) )
            else:
                ups.append( (xs, (np.array([0]))) )

        #if (i%1000)==0:
        #    clear_output(wait=True)
        #    print('({}/{})'.format(i, input_source.shape[1]-winlen))

    #print('Ups: {0} Mids: {1} Downs: {2}'.format(len(ups), len(mids), len(downs)))
    #print('Separator: %3.5f' % sep)
    #print()

    if not binary:
        samplesize = min(len(ups), len(downs), len(mids))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        s3 = rnd.sample(mids, samplesize)
        a = s1 + s2 + s3
    else:
        samplesize = min(len(ups), len(downs))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        a = s1 + s2 
    rnd.shuffle(a)
    x = [x[0] for x in a]
    y = [x[1] for x in a]

    x = np.vstack(x)
    y = np.vstack(y)
    
    x.shape = (x.shape[0], -1)
    cutpoint = int(0.8 * x.shape[0])
    x_train = x[0:cutpoint]
    x_test = x[cutpoint:]
    y_train = y[0:cutpoint]
    y_test = y[cutpoint:]
    
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1)

In [40]:
x_train, x_test, y_train, y_test = prepare_data(input_source, to_predict)

In [41]:
#x_train[0:10]

In [42]:
feature_names = ['feature_'+str(i) for i in range(input_source.shape[0])]
skr = SkopeRules(max_depth_duplication=2,
                 n_estimators=100,
                 precision_min=0.3,
                 recall_min=0.1,
                 n_jobs=8,
                 feature_names=feature_names)

In [43]:
skr.fit(x_train, y_train)
rules = skr.rules_#[0:10]
print("Rules for dataset:")
for rule in rules:
    print(rule)
print()

In [44]:
dfx = np.zeros(x_test.shape[0])

for r in rules[0:5]:
    rule = r[0]
    df = pd.DataFrame(x_test, columns=feature_names)
    df = df.reset_index(drop=True)
    dfx[list(df.query(rule).index)] = 1

dfx

In [45]:
hist(dfx);

In [22]:
sum(dfx == y_test) / len(y_test)

In [23]:
y_test

In [24]:
sum([x == y for x,y in zip(dfx, y_test)])/len(y_test)

In [25]:
def predict_rules(rules, x):
    dfx = np.zeros(x.shape[0])

    for r in rules:
        rule = r[0]
        df = pd.DataFrame(x, columns=feature_names)
        df = df.reset_index(drop=True)
        dfx[list(df.query(rule).index)] = 1
        
    return dfx

In [26]:
input_source.shape, to_predict.shape, test_input_source.shape, test_to_predict.shape

In [27]:
traded_amt = 100
commission = 0
slippage = 0
initial_balance = 100000000

In [28]:
def plot_backtest(datapair, clf): 
    input_source, to_predict = datapair
    bars_per_episode = len(to_predict) - 1000
    env = TradingEnv(input_source.T, to_predict,
                     winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt,
                     commission=commission, slippage=slippage,
                     initial_balance=initial_balance,
                     reward_type='cur_balance',
                     min_ratio_trades = 20,
                     max_position_time = 0,
                     )
    observation = env.reset()
    done = False
    navs = []
    acts = []
    for i in tqdm(range(env.bars_per_episode)):
        #a = clf.predict([observation])
        a = predict_rules(rules[0:5], array([observation]))
        a = int(a)
        if not binary:
            if   a == 0: # up
                action = 0 # buy
            elif a == 1: # mid 
                action = 3 # do nothing
            elif a == 2: # down
                action = 1 # sell
        else:
            if   a == 0: # up
                action = 0 # buy
            elif a == 1: # down
                action = 1 # sell
        acts.append(action)
        observation, reward, done, info = env.step(action)
        if done:
            break
    navs = env.returns
    kl = []
    t = 0
    for n in navs:
        t = t + n
        kl.append(t)
    plt.plot(kl)
    plt.show()

In [29]:
plot_backtest((input_source, to_predict), clf)

In [30]:
plot_backtest((test_input_source, test_to_predict), clf)